Repo to create multiple example for Rasa Intents and put paraphrases into nlu.yml file to augment the training set of examples for intents.

In [ ]:
!pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

In [3]:
# Use Huggingface Hub for paraphrasing model
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
# Use Parrot Paraphraser
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

#Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5")

In [5]:
# Upload dev_light.json file for question/answers
from google.colab import files

uploaded = files.upload()

In [6]:
from dataclasses import replace
import json
import re 

In [7]:
# Parse the JSON dataset file and put questions, answers and ids into arrays
questions = []
answers = []
ids = []
count = 0

# Opening JSON file
f = open('dev_light.json', encoding="utf8")
  
# returns JSON object as 
# a dictionary
data = json.load(f)

# Use on SingleAnswer questions from the database for starter
for i in data:
  if i["annotations"][0]["type"] == 'singleAnswer':
    count += 1
    # Make questions all lowercase
    questions.append(i["question"].lower())
    answers.append(i["annotations"][0]["answer"][0])
    # Remove - sign from all intents
    ids.append(re.sub("^-", "", i["id"]))  

print(f"Total number of single answers: {count}")



Total number of single answers: 932


In [ ]:
# Create nlu.yml file
count = -1
validcount = 0
intent = ""
curphrase = ""
curparaphrase = ""
validphrases = []

with open('nlu.yml', 'w') as nlufile:
  nlufile.write('version: "3.1"')
  nlufile.write("\n")
  nlufile.write("nlu:\n")

  # include default questions
  nlufile.write("- intent: greet\n")
  nlufile.write("  examples: |\n")
  nlufile.write("    - hey\n")
  nlufile.write("    - hello\n")
  nlufile.write("    - hi\n")
  nlufile.write("    - hello there\n")
  nlufile.write("    - good morning\n")
  nlufile.write("    - good evening\n")
  nlufile.write("    - moin\n")
  nlufile.write("    - hey there\n")
  nlufile.write("    - let's go\n")
  nlufile.write("    - hey dude\n")
  nlufile.write("    - goodmorning\n")
  nlufile.write("    - goodevening\n")
  nlufile.write("    - good afternoon\n")

  nlufile.write("- intent: goodbye\n")
  nlufile.write("  examples: |\n")
  nlufile.write("    - cu\n")
  nlufile.write("    - good by\n")
  nlufile.write("    - cee you later\n")
  nlufile.write("    - good night\n")
  nlufile.write("    - bye\n")
  nlufile.write("    - goodbye\n")
  nlufile.write("    - have a nice day\n")
  nlufile.write("    - see you around\n")
  nlufile.write("    - bye bye\n")
  nlufile.write("    - see you later\n")

  nlufile.write("- intent: affirm\n")
  nlufile.write("  examples: |\n")
  nlufile.write("    - yes\n")
  nlufile.write("    - y\n")
  nlufile.write("    - indeed\n")
  nlufile.write("    - of course\n")
  nlufile.write("    - that sounds good\n")
  nlufile.write("    - correct\n")

  nlufile.write("- intent: deny\n")
  nlufile.write("  examples: |\n")
  nlufile.write("    - no\n")
  nlufile.write("    - n\n")
  nlufile.write("    - never\n")
  nlufile.write("    - I don't think so\n")
  nlufile.write("    - don't like that\n")
  nlufile.write("    - no way\n")
  nlufile.write("    - not really\n")

  nlufile.write("- intent: mood_great\n")
  nlufile.write("  examples: |\n")
  nlufile.write("    - perfect\n")
  nlufile.write("    - great\n")
  nlufile.write("    - amazing\n")
  nlufile.write("    - feeling like a king\n")
  nlufile.write("    - wonderful\n")
  nlufile.write("    - I am feeling very good\n")
  nlufile.write("    - I am great\n")
  nlufile.write("    - I am amazing\n")
  nlufile.write("    - I am going to save the world\n")
  nlufile.write("    - super stoked\n")
  nlufile.write("    - extremely good\n")
  nlufile.write("    - so so perfect\n")
  nlufile.write("    - so good\n")
  nlufile.write("    - so perfect\n")

  nlufile.write("- intent: mood_unhappy\n")
  nlufile.write("  examples: |\n")
  nlufile.write("    - my day was horrible\n")
  nlufile.write("    - I am sad\n")
  nlufile.write("    - I don't feel very well\n")
  nlufile.write("    - I am disappointed\n")
  nlufile.write("    - super sad\n")
  nlufile.write("    - I'm so sad\n")
  nlufile.write("    - sad\n")
  nlufile.write("    - very sad\n")
  nlufile.write("    - unhappy\n")
  nlufile.write("    - not good\n")
  nlufile.write("    - not very good\n")
  nlufile.write("    - extremly sad\n")
  nlufile.write("    - so saad\n")
  nlufile.write("    - so sad\n")

  nlufile.write("- intent: bot_challenge\n")
  nlufile.write("  examples: |\n")
  nlufile.write("    - are you a bot?\n")
  nlufile.write("    - are you a human?\n")
  nlufile.write("    - am I talking to a bot?\n")
  nlufile.write("    - am I talking to a human?\n")

  for question in questions:
    count = count + 1
    print(count)
    para_phrases = parrot.augment(input_phrase=question, use_gpu=False)
    # Use the question only if the paraphraser returns at least one paraphrase of the question
    if para_phrases and len(para_phrases) > 1:
      print(f"{count} {ids[count]} {questions[count]} {answers[count]}")
      # Mark this intent as valid
      validcount = validcount + 1
      validphrases.append(ids[count])
      intent = "- intent: name{}\n".format(ids[count])
      nlufile.write(intent)
      nlufile.write("  examples: |\n")
      curphrase = "    - {}\n".format(questions[count])
      nlufile.write(curphrase)
      for para_phrase in para_phrases:
        curparaphrase = "    - {}\n".format(para_phrase[0])
        nlufile.write(curparaphrase)
    

print(f"Total valid intents: {validcount} of {count} questions\n")

In [9]:
# Create stories.yml file
curstory = ""
count = -1
with open('stories.yml', 'w') as stories:
  stories.write('version: "3.1"')
  stories.write("\n")
  stories.write("\n")
  stories.write("stories:\n\n") 

  # include the default stories
  stories.write("- story: happy path\n")
  stories.write("  steps:\n")
  stories.write("  - intent: greet\n")
  stories.write("  - action: utter_greet\n")
  stories.write("  - intent: mood_great\n")
  stories.write(" - action: utter_happy\n")

  stories.write("- story: sad path 1\n")
  stories.write("  steps:\n")
  stories.write("  - intent: greet\n")
  stories.write("  - action: utter_greet\n")
  stories.write("  - intent: mood_unhappy\n")
  stories.write("  - action: utter_cheer_up\n")
  stories.write("  - action: utter_did_that_help\n")
  stories.write("  - intent: affirm\n")
  stories.write("  - action: utter_happy\n")

  stories.write("- story: sad path 2\n")
  stories.write("  steps:\n")
  stories.write("  - intent: greet\n")
  stories.write("  - action: utter_greet\n")
  stories.write("  - intent: mood_unhappy\n")
  stories.write("  - action: utter_cheer_up\n")
  stories.write("  - action: utter_did_that_help\n")
  stories.write("  - intent: deny\n")
  stories.write("  - action: utter_goodbye\n")

  for question in questions:
    count = count + 1
    # Include on valid questions (has more than 1 example)
    if ids[count] in validphrases:
      stories.write("- story: {}\n".format(questions[count]))
      stories.write("  steps:\n")
      stories.write("  - intent: ")
      stories.write("name{}\n".format(ids[count]))
      stories.write("  - action: utter_name{}\n\n".format(ids[count]))

In [12]:
# Create domain.yml file
count = -1

with open('domain.yml', 'w') as domains:
  domains.write('version: "3.1"\n')
  domains.write("\n")
  domains.write("intents:\n")

  # Include default intents
  domains.write("  - greet\n")
  domains.write("  - goodbye\n")
  domains.write("  - affirm\n")
  domains.write("  - deny\n")
  domains.write("  - mood_great\n")
  domains.write("  - mood_unhappy\n")
  domains.write("  - bot_challenge\n")
  
  for question in questions:
    count = count +1
    # Only include valid questions
    if ids[count] in validphrases:
      domains.write("  - name{}\n".format(ids[count]))
  domains.write("\n")
  domains.write("responses:\n")
  
  # Include default responses
  domains.write("  utter_greet:\n")
  domains.write("  - text: \"Hey! How are you?\"\n")
  domains.write("\n")
  domains.write("  utter_cheer_up :\n")
  domains.write("  - text: \" Here is something to cheer you up:\"\n")
  domains.write("  image: \"https://i.imgur.com/nGF1K8f.jpg\"\n")
  domains.write("\n")
  domains.write("  utter_did_that_help:\n")
  domains.write("  - text: \"Did that help you?\"\n")
  domains.write("\n")
  domains.write("  utter_happy:\n")
  domains.write("  - text: \"Great, carry on!\"\n")
  domains.write("\n")
  domains.write("  utter_goodby:\n")
  domains.write("  - text: \"Bye\"\n")
  domains.write("\n")
  domains.write("  utter_iamabot:\n")
  domains.write("  - text: \"I am a bot, powered by Rasa.\"\n")
  domains.write("\n")
  
  count = -1
  for question in questions:
    count = count + 1
    # include only utterances that have valid intents
    if ids[count] in validphrases:
      domains.write("  utter_name{}:\n".format(ids[count]))
      domains.write("  - text: \"{}\"\n".format(answers[count]))
      domains.write("\n")
  domains.write("\n")
  domains.write("session_config:\n")
  domains.write("  session_expiration_time: 60\n")
  domains.write("  carry_over_slots_to_new_session: true\n")
